In [ ]:
# experimento com as imagens de todas as coletas da Palmer, utilizando 66% para treinamento e 33% para teste, para predição de SST

In [8]:
import cv2
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os 
import math

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split

%load_ext autoreload
plt.rcParams['figure.figsize'] = [20, 15]

path = os.getcwd()+'//..//imagens//'
sys.path.append(os.getcwd()+'//..//scripts//')
from MNG import MNG
from MNGFeatures import MNGFeatures
from MNGFeaturesMeans import MNGFeaturesMeans
from MNGModel import MNGModel

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
%autoreload 2

att = 'sst'
model_type = 'palmer'
palmer_sst = path + '\\..\\palmer_sst.csv'

In [35]:
path = os.getcwd() + '//..//preprocessadas2//'
imgs = os.listdir(path)
img_names = [img.split('.')[0] for img in imgs]

mng = MNG(path, img_names, 'time', 20)

features_mlr = mng.features.feature_names[13]
mng.features.current_features = features_mlr
mng.features.current_features_name = 'dominant_HSV'
mng.features.new_df()

In [36]:
mng.features.dest_folder = os.getcwd() + '//..//features2//'

In [37]:
nums = np.arange(1, 180)

for img,num in zip(imgs, nums):
    print(num)
    bgr = cv2.imread(path+img)
    mng.features.extract_features(bgr, img)
    
file_path = mng.features.save_data(False)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179


In [23]:
file_path

'/home/danielesreis/Projects/mng/notebooks//..//preprocessadas2//..\\features\\means_all.csv'

In [27]:
mng.features.add_target(file_path, palmer_sst)

In [42]:
mng.model = MNGModel(path, file_path, model_type, att)
mng.model.build_mlr_model(model_type, features_mlr)

trees = [100, 200, 300, 400, 500, 600, 700]
for tree_num in trees:
    mng.model.build_rf_model(model_type, tree_num)

[0.01101535 0.01105941 0.89462016 0.01667592 0.01484859 0.00772028
 0.00612206 0.03145937 0.00647887]
[0.01528925 0.01133236 0.86263109 0.02499515 0.01571459 0.00982194
 0.0136343  0.03835928 0.00822203]
[0.01565045 0.01643335 0.82054462 0.03230305 0.02567242 0.00923339
 0.01624256 0.04844877 0.0154714 ]
[0.01341815 0.01364319 0.86405266 0.03122513 0.01700749 0.00904529
 0.01445426 0.02677317 0.01038067]
[0.01588813 0.01426283 0.8187836  0.02980261 0.02829098 0.0102489
 0.01157976 0.06055222 0.01059097]
[0.01366671 0.01975969 0.81605826 0.02393945 0.04499    0.00867667
 0.0151067  0.04395046 0.01385206]
[0.01024313 0.01259795 0.87850702 0.02085681 0.01354128 0.00899067
 0.01263427 0.03261116 0.01001772]


In [ ]:
# TESTE COM RGB APENAS

In [84]:
df_rgb = mng.model.data[['sst', 'mean_R_full', 'mean_G_full', 'mean_B_full']].copy()
df_rgb_path = path+'\\..\\features\\apenas_rgb.csv'
df_rgb.to_csv(df_rgb_path, sep=';')

In [75]:
mng.model = MNGModel(path, df_rgb_path, model_type, att)
mng.model.build_mlr_model(model_type, features_mlr)

trees = [100, 200, 300, 400, 500, 600, 700]
for tree_num in trees:
    mng.model.build_rf_model(model_type, tree_num)

[0.06636356 0.04893261 0.88470383]
[0.06429572 0.06301666 0.87268762]
[0.08230356 0.04243272 0.87526371]
[0.07087914 0.04359687 0.88552398]
[0.05384543 0.05248197 0.8936726 ]
[0.04399283 0.04615855 0.90984862]
[0.08021652 0.04966366 0.87011982]


In [ ]:
# TESTE COM HSV APENAS

In [81]:
df_hsv = mng.model.data[['sst', 'mean_H_full', 'mean_S_full', 'mean_V_full']].copy()
df_hsv_path = path+'\\..\\features\\apenas_hsv.csv'
df_hsv.to_csv(df_hsv_path, sep=';')

In [82]:
mng.model = MNGModel(path, df_hsv_path, model_type, att)
mng.model.build_mlr_model(model_type, features_mlr)

trees = [100, 200, 300, 400, 500, 600, 700]
for tree_num in trees:
    mng.model.build_rf_model(model_type, tree_num)

[0.06949251 0.7438529  0.18665459]
[0.07760629 0.7690112  0.15338251]
[0.05819901 0.78665167 0.15514932]
[0.05050865 0.80939203 0.14009932]
[0.05386391 0.78753751 0.15859858]
[0.05552808 0.79493086 0.14954107]
[0.05391616 0.77063927 0.17544457]


In [ ]:
# TESTE SEM SEMANA 4

In [53]:
sem_4 = mng.model.data.copy()
inds = mng.model.data.loc['0361palmer_sem4_num1':'0479palmer_sem4_num60'].index.values
sem_4.drop(inds, inplace=True)

sem_4_path = path+'\\..\\features\\sem_4.csv'
sem_4.to_csv(sem_4_path, sep=';')

In [54]:
mng.model = MNGModel(path, sem_4_path, model_type, att)
mng.model.build_mlr_model(model_type, features_mlr)

trees = [100, 200, 300, 400, 500, 600, 700]
for tree_num in trees:
    mng.model.build_rf_model(model_type, tree_num)